In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# read in pickle
with open('result2.pkl', 'rb') as f:
    result = pickle.load(f)

# convert to dataframe and store as pickle
df = pd.DataFrame(result)
df.to_pickle('df.pkl')

# print df head
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,86ff1ea1-0b63-43ce-addc-eb43f6193b3b,Yaseen Yaseen,https://play-lh.googleusercontent.com/a/ALm5wu...,Yaeen Yaeen gg,5,0,None,2022-10-04 20:32:28,None,NaT
1,3577f7a1-3394-4e77-813d-095a82cf8bcf,Kemar Richardson,https://play-lh.googleusercontent.com/a-/ACNPE...,Great,5,0,26.3.4,2022-10-04 20:32:10,None,NaT
2,7c8c56d9-d8ad-47d4-b24b-5289aa4529ff,Tracy Dunn,https://play-lh.googleusercontent.com/a/ALm5wu...,good,5,0,26.4.3,2022-10-04 20:31:21,None,NaT
3,80db804f-cccd-4b09-b690-abc12cbf0612,SG. Mugo. (Mugoz:),https://play-lh.googleusercontent.com/a-/ACNPE...,Good app,5,0,26.3.4,2022-10-04 20:30:22,None,NaT
4,4ed35e90-0f45-4865-81c4-b3a6f2ea49f7,Mwansa Judy,https://play-lh.googleusercontent.com/a-/ACNPE...,Most amazing app,5,0,26.3.4,2022-10-04 20:29:25,None,NaT


In [3]:
# check fields inside df
df.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt'],
      dtype='object')

In [4]:
# check how many reviews with score of 1
df[df.score == 1].count()

reviewId                14178
userName                14178
userImage               14178
content                 14178
score                   14178
thumbsUpCount           14178
reviewCreatedVersion     9491
at                      14178
replyContent               13
repliedAt                  13
dtype: int64

In [5]:
import pandas as pd
from pandas import DataFrame
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

We will implement functions to clean up the tweets, we will want to remove unnecesary stop words, punctuations @mentions, #hashtags, emojis and lemmatize the tweets

In [6]:
def remove_mentions(tweet: str) -> str:
    clean_tweet: str = re.sub("@[A-Za-z0-9_]+", "", tweet)
    return clean_tweet
    
def remove_hash_tag(tweet: str) -> str:
    clean_tweet: str = re.sub("#[A-Za-z0-9_]+", "", clean_tweet)
    return clean_tweet


def remove_emojis(tweet: str) -> str:
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

def remove_punctuations(review: str) -> str:
    words: list[str] = review.split()
    table: dict[int, int] = str.maketrans('', '', string.punctuation)
    stripped: list[str] = [w.translate(table) for w in words]
    return " ".join(stripped)

def normalize(review: str) -> str:
    return review.lower()

def remove_stopwords(review: str, stopwords: set[str]) -> str:
    words: list[str] = review.split()
    words = [w for w in words if not w in stopwords]
    return " ".join(words)


def stemming(review: str) -> str:
    tokens: list[str] = word_tokenize(review)
    porter: PorterStemmer = PorterStemmer()
    stemmed: list[str] = [porter.stem(word) for word in tokens]
    return " ".join(stemmed)

def lemmatize(review: str) -> str:
    lemmatizer = WordNetLemmatizer()
    tokens: list[str] = word_tokenize(review)
    lemmatized: list[str] = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

In [7]:
def clean_review(review: str, stopwords: set[str]) -> str:
    result: str = remove_stopwords(normalize(remove_punctuations(review)), stopwords)
    return result

In [8]:
def clean_data(df: DataFrame):
    # df['content'] = df['content'].apply(lambda x: x if is_english(x) else '')
    # df = df[df['content'] != '']
    review: list[str] = list(df["content"])
    stop_words: set[str] = set(stopwords.words('english'))
    cleaned_review: list[str] = list()
    for i, review in enumerate(review):
        cleaned_review = clean_review(review, stop_words)
        cleaned_review.append(cleaned_review)
    df["Clean_texts"] = cleaned_review

In [9]:
# from langdetect import detect
# def detect_my(text):
#    try:
#        return detect(text)
#    except:
#        return 'unknown'

# # df['language'] = df['content'].apply(detect_my)

# detect_my('english')

In [10]:
import re

# return new column "is_english" with True or False
def is_english(text):
    try:
        return bool(re.search('[a-zA-Z]', text))
    except:
        return False

df['is_english'] = df['content'].apply(is_english)

# remove puncutation
df['content'] = df['content'].str.replace('[^\w\s]','')

# save new df as pickle
df.to_pickle('df_english.pkl')

# save df to json
df.to_json('df_english.json')

# save df to csv
df.to_csv('df_english.csv')

C:\Users\ngliw\AppData\Local\Temp\ipykernel_18336\165540621.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]','')
